# Caso

# Análisis (analysis)
Las secciones Imports y System parameters siempre deben ejecutarse antes de ejecutar cualquier otra sección (menos una de las celdas de System parameters, que solo es necesario ejecutarla la primera vez que se usa el cuaderno).<br />
El resto de secciones son independientes de la ejecución de las otras secciones.

## Index
0. [Imports](#Imports)
1. [System parameters](#System-parameters)
2. [Water and chloride inside the region](#Water-and-chloride-inside-the-region)
3. [Hidrogen bonds](#Hydrogen-bonds)
4. [Hydrogen bond statistics](#Hydrogen-bond-statistics)
5. [Hydrogen bond stability](#Hydrogen-bond-stability)
6. [Hydrogen bond "paths"](#Hydrogen-bond-"paths")
7. [Hydrogen bond temporal correlation](#Hydrogen-bond-temporal-correlation)
8. [Hydrogen bond orientation](#Hydrogen-bond-orientation)
9. [Atoms' average positions](#Atoms'-average-positions)
10. [Radial Distribution Function and "Z" Distribution Function](#Radial-Distribution-Function-and-"Z"-Distribution-Function)
11. [Coordination numbers](#Coordination-numbers)
12. [Closest atoms](#Closest-atoms)

## Imports

In [ ]:
import sys
sys.path.insert(1, '/home/jorge/analisis/main/')
# sys.path.append('/ufs/guido/lib/python')
from main import *
from analisis import *

## System parameters
Inicializamos los parámetros del sistema, algunos explícitamente y otros dentro de un `Params`.<br />
La función `recenter_traj_RMSD` debe ser ejecutada una vez parama crear la trayectoria `traj_name_RMSD.nc` con los nanotubos centrados y alineando su eje con el eje Z del sistema de coordenadas.

In [ ]:
# Parámetros fijos del sistema
'''
Los tubos están colocados:
1 2
3 4
'''

traj_name = "4tubes_run01"
N_tubes = 4 # Número de tubos en el sistema
N_rings = 6 # Número de anillos en un tubo
N_res = 8 # Número de residuos en un anillo
# Los índices de los residuos (LYS y LYN) que están en el canal entre los 4 nanotubos son (el primer índice es 1):
# 3 15 19 31 35 47 53 61 69 77 85 93 101 109 117 125 133 141 147 159 163 175 179 191
channel_res_1 = "3, 15, 19, 31, 35, 47"
channel_res_2 = "53, 61, 69, 77, 85, 93"
channel_res_3 = "101, 109, 117, 125, 133, 141"
channel_res_4 = "147, 159, 163, 175, 179, 191"

In [ ]:
# ONLY ONE TIME
# recenter_traj_RMSD(traj_name, N_tubes, N_res)

In [ ]:
traj = pt.iterload(traj_name+"_RMSD.nc", traj_name+".top")
p = Params(traj, N_tubes, N_rings, N_res, channel_res_1, channel_res_2, channel_res_3, channel_res_4)

## Water and chloride inside the region
La función `get_indices` guarda dos archivos .npy con los índices de (los oxígenos de) las aguas y los cloros que hay en la región de interés.<br />
Estos archivos se leen con `indices = np.load("file.npy", allow_pickle=True)`. `indices[step]` es un array con los índices de los átomos (oxígenos de aguas o cloros) en la región de interés en cierto `step` de la simulación.<br />
Por defecto la función está preparada para una primera llamada en la que guarda los índices de los átomos en una región amplia alrededor de los 4 nanotubos. Después, en una llamada posterior guardaría los índices de los átomos en una región más pequeña (por ejemplo, el canal) dentro de la primera región ampia. (De esta forma solo tiene que comprobar si los átomos de la región amplia están también dentro de la región pequeña, en vez de tener que hacer la comprobación para todos los átomos de la simulación)<br />
<span style="color:red">Estaría bien poder definir dos regiones: región "proper" y región frontera. Los átomos en la región frontera se tendrían en cuenta a la hora de definir los puentes de hidrógeno / números de coordinación de los átomos de la región "proper", pero no se definiría ningún puente de hidrógeno entra átomos en la región frontera. De esta forma evitaríamos "efectos de borde" introducidos artificialmente en el análisis al seleccionar la región.</span>

In [ ]:
# Guardar los índices de CLs y WATs en una región amplia alrededor de todos los tubos
get_indices(traj, p.WATs, p.CLs, p.CAs_topbot, N_res*N_tubes)

In [ ]:
# Guardar los índices de CLs y WATs dentro de los tubos y del canal

iWATs = np.load("iWATs.npy", allow_pickle=True)
iCLs = np.load("iCLs.npy", allow_pickle=True)

print("Analizando tubo 1")
get_indices(traj, iWATs, iCLs, p.CAs_tube1, N_res, layer=1,
            delta=0.0, preselected=True, save=True, savefileWATs="iWATs_tube1", savefileCLs="iCLs_tube1")

# print("Analizando tubo 2")
# get_indices(traj, iWATs, iCLs, p.CAs_tube2[N_res:2*N_res], p.CAs_tube2[-2*N_res:-N_res],
#             delta=0.0, preselected=True, save=True, savefileWATs="iWATs_tube2.npy", savefileCLs="iCLs_tube2.npy")

# print("Analizando tubo 3")
# get_indices(traj, iWATs, iCLs, p.CAs_tube3[N_res:2*N_res], p.CAs_tube3[-2*N_res:-N_res],
#             delta=0.0, preselected=True, save=True, savefileWATs="iWATs_tube3.npy", savefileCLs="iCLs_tube3.npy")

# print("Analizando tubo 4")
# get_indices(traj, iWATs, iCLs, p.CAs_tube4[N_res:2*N_res], p.CAs_tube4[-2*N_res:-N_res],
#             delta=0.0, preselected=True, save=True, savefileWATs="iWATs_tube4.npy", savefileCLs="iCLs_tube4.npy")

print("Analizando canal")
get_indices(traj, iWATs, iCLs, p.CAs_canal, N_tubes, layer=1,
            delta=0.0, preselected=True, save=True, savefileWATs="iWATs_canal", savefileCLs="iCLs_canal")

## Hydrogen bonds
La función `analyse` analiza los enlaces de tipo puente de hidrógeno en la región de interés.
Esta función crea dos archivos .csv:<br />
`label_stats.csv` contiene las estadísticas de número de aguas, número de cloros, número de puentes de hidrógeno (y también la distancia media de estos puentes de hidrógeno) que hay en la región en cada frame. En el cuaderno `new-estadisticas.ipynb` se representan estas cantidades.<br />
`label_hbonds.csv` contiene la información de todos los enlaces de puente de hidrógeno que ha encontrado la función: frame en el que se da el enlace, donor, H del donor, acceptor, donor residue - acceptor residue, distancia del enlace.<br />
<span style="color:red">Lo dicho, sería interesante tener en cuenta dos regiones.</span>

In [ ]:
# Analizar puentes de hidrógeno dentro de los tubos y el canal
# (Esto tarda bastante)

print("Analizando tubo 1")
analyse(p, traj, "tube1", layer=1)

# print("Analizando tubo 2")
# iWATs_tube2 = np.load("iWATs_tube2.npy", allow_pickle=True)
# iCLs_tube2 = np.load("iCLs_tube2.npy", allow_pickle=True)
# analyse(p, traj, iWATs_tube2, iCLs_tube2, "tube2")

# print("Analizando tubo 3")
# iWATs_tube3 = np.load("iWATs_tube3.npy", allow_pickle=True)
# iCLs_tube3 = np.load("iCLs_tube3.npy", allow_pickle=True)
# analyse(p, traj, iWATs_tube3, iCLs_tube3, "tube3")

# print("Analizando tubo 4")
# iWATs_tube4 = np.load("iWATs_tube4.npy", allow_pickle=True)
# iCLs_tube4 = np.load("iCLs_tube4.npy", allow_pickle=True)
# analyse(p, traj, iWATs_tube4, iCLs_tube4, "tube4")

print("Analizando canal")
analyse(p, traj, "canal", canal=True, layer=1)

## Hydrogen bond statistics
La función `detail_hbonds` lee el archivo `label_hbonds.csv` y crea el archivo `label_hbonds_detail.csv`. Este archivo contiene la información sobre cuántos enlaces de puente de hidrógeno de cada tipo (i.e. WAT-WAT, LYS-WAT, etc) se dan en cada frame y cuál es su distancia media.

In [ ]:
detail_hbonds("canal")

In [ ]:
detail_hbonds("tube1")

## Hydrogen bond stability
La función `stability_hbonds` lee el archivo `label_hbonds.csv` y crea el archivo `label_hbonds_stability.csv`. Este archivo recoge en cuántos frames consecutivos aparece cada enlace de puente de hidrógeno.

In [ ]:
stability_hbonds("canal")

In [ ]:
stability_hbonds("tube1")

## Hydrogen bond "paths"
La función `save_paths` lee el archivo `label_hbonds.csv` y la trayectoria para crear el archivo `label_paths.csv`. Este archivo indica, para cada frame de la simulación, cuál es el "path" de puentes de hidrógeno que abarca mayor distancia en el eje Z.

In [ ]:
save_paths(traj, "canal")

In [ ]:
save_paths(traj, "tube1")

## Hydrogen bond temporal correlation
La función `hbonds_correlation` lee el archivo `label_hbonds.csv` y crea un gráfico de la correlación de los puentes de hidrógeno. La correlación se calcula de la siguiente forma:<br />
$ \eta(t) = \sum_k \delta_{H_k(t), H_k(0)} $,<br />
donde $ H_k(t) $ es el índice del átomo con el que puentea el hidrógeno $ k $, y que hacemos $ 0 $ si no puentea con ningún átomo.<br />
$ \delta_{H_k(t), H_k(0)} = 1 $ si $ H_k(t) = H_k(0) \neq 0 $ y es $ 0 $ en caso contrario.<br />

In [ ]:
fig, ax = hbonds_correlation("canal")
title = 'Hydrogen bond correlation inside the channel'
xlabel = 'Simulation time (ns)'
ylabel = 'Correlation'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = hbonds_correlation("tube1")
title = 'Hydrogen bond correlation inside the tube'
xlabel = 'Simulation time (ns)'
ylabel = 'Correlation'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

## Hydrogen bond orientation

In [ ]:
fig, ax = hbonds_orientation(traj, "canal")
title = 'Hydrogen bond orientation inside the channel'
xlabel = r'$\theta (^\circ)$'
ylabel = 'Counts'
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = hbonds_orientation(traj, "canal", bondtype="WAT-WAT")
title = 'Hydrogen bond orientation inside the channel (only WAT-WAT)'
xlabel = r'$\theta (^\circ)$'
ylabel = 'Counts'
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = hbonds_orientation(traj, "tube1")
title = 'Hydrogen bond orientation inside the tube'
xlabel = r'$\theta (^\circ)$'
ylabel = 'Counts'
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

## Atoms' average positions
La función `average_positions` toma la trayectoria de la simulación y calcula las posiciones medias de los átomos en la región de interés. El archivo `label_averages.csv` que escribe contiene la información sobre el número que aparece cierto átomo en la región de interés, su posición media y la varianza de esta.

In [ ]:
average_positions(p, traj, "canal", canal=True, layer=0)

In [ ]:
average_positions(p, traj, "tube1", layer=0)

## Radial Distribution Function and "Z" Distribution Function
La función `RDF` computa y grafica los histogramas del conteo de las veces en las que los de átomos de un tipo que se encuentran a cierta distancia de los átomos de otro tipo.<br />
Una de las curvas (`r`) representa el histograma de la distancia euclidiana en 3D entre los átomos, y la otra curva (`z`) representa el histograma de la proyección en el eje Z de la distancia entre átomos.

In [ ]:
fig, axs = RDF(p, traj, "canal", layer=0)
xlabel = 'Distance ($\AA$)'
ylabel = 'Count'
fig.supxlabel(xlabel, fontsize=14)
fig.supylabel(ylabel, fontsize=14)
for ax in axs.flatten():
    decorate_ax(ax, ax.get_title(), 14, '', ax.get_ylabel(), 14, 12, 2, 4, True)

## Coordination numbers
La función `coordination` escribe el archivo `label_coordination.csv`, que contiene la información de cuántos átomos de cada tipo se encuentran alrededor de cada átomo en cada frame.<br/ >
La función `coordination_hbonds` escribe el archivo `label_coordination_hbonds.csv`, que contiene la información de cuántos puentes de hidrógeno de cada tipo (con qué atomo y si es donor o acceptor) está formando cada átomo en cada frame.

In [ ]:
coordination(p, traj, "canal", layer=0)

In [ ]:
coordination_hbonds(p, traj, "canal", layer=0)

## Closest atoms
Las funciones `closest_atoms` y `plot_closest_atoms` sirven para representar cuál es el átomo de un cierto tipo que más cerca está de un átomo en concreto.

In [ ]:
iCLs_canal = np.load("iCLs_canal.npy", allow_pickle=True)
aux = np.array([], dtype=int)
for CLs in iCLs_canal:
    aux = np.append(aux, CLs)
inCLs = np.unique(aux)
print(len(inCLs))
# for index in range(len(inCLs)):
#     print(inCLs[index])

iWATs_canal = np.load("iWATs_canal.npy", allow_pickle=True)
aux = np.array([], dtype=int)
for WATs in iWATs_canal:
    aux = np.append(aux, WATs)
inWATs = np.unique(aux)
print(len(inWATs))
# for index in range(len(inWATs)):
#     print(inWATs[index])

NZs_LYS, NZs_LYN = select_atoms(p, 0)
NZs = np.concatenate((NZs_LYS, NZs_LYN))

In [ ]:
distances, bonds = closest_atoms(traj, inCLs, iCLs_canal, NZs)

In [ ]:
fig, ax = plot_closest_atoms(distances, bonds, 1)
title = 'Distance to the closest atom'
xlabel = 'Simulation time (ns)'
ylabel = 'Distance ($\AA$)'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = plot_distance(traj, inCLs, 1, 2423)
title = 'Distance between two atoms'
xlabel = 'Simulation time (ns)'
ylabel = 'Distance ($\AA$)'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for index in range(len(inCLs)):
    zs = []
    for step in range(0, len(traj)):
        frame = traj[step]
        zs.append(frame[inCLs[index]][2])
    ax.plot(np.linspace(0, 100, len(traj)), zs, label=str(inCLs[index]), lw=1)
ax.legend(edgecolor='0.75')

title = 'Z coordinate of chlorides inside the channel'
xlabel = 'Simulation time (ns)'
ylabel = 'Z coordinate ($\AA$)'
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, True)

In [ ]:
distances, bonds = closest_atoms(traj, inWATs, iWATs_canal, NZs)

In [ ]:
for index in range(len(inWATs)):
    if len(bonds[index][bonds[index]>0]) > 1000:
        print(index)

In [ ]:
fig, ax = plot_closest_atoms(distances, bonds, 104)
title = 'Distance to the closest atom'
xlabel = 'Simulation time (ns)'
ylabel = 'Distance ($\AA$)'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)

In [ ]:
fig, ax = plot_distance(traj, inWATs, 104, 2589)
title = 'Distance between two atoms'
xlabel = 'Simulation time (ns)'
ylabel = 'Distance ($\AA$)'
loc = ax.get_xticks()
ax.set_xticks(loc[1:-1], (100*loc[1:-1]/len(traj)).astype(int))
decorate_ax(ax, title, 16, xlabel, ylabel, 14, 12, 2, 4, False)